# Datojen haku ja esikäsittely

In [1]:
from set_path import set_path
mainpath, path = set_path('areadata')

In [2]:
from read_and_prepare_data import read_and_prepare_data
stat, post, kunta_stat, vaalidata = read_and_prepare_data(path)

read_and_prepare_data  read from file
read_post_muncipalities  read from file


In [3]:
from selected_cols import selected_cols
numeric_features, categorical_features = selected_cols(largeset=False, parties=False)

In [4]:
from sklearn.feature_selection import  mutual_info_classif
import pandas as pd

from draw_and_create_clusters import create_kmeans_clusters 

from prepare_and_scale_data import prepare_and_scale_data
from create_prediction import select_kbest
from select_columns_and_clean_data import select_columns_and_clean_data

from draw_and_create_clusters import draw_pca, drawTSNE,  display_scree_plot, display_circles, display_parallel_coordinates_centroids, display_factorial_planes
from delete_outliers import delete_outliers

from shap_Xboost import shap_Xboost
from shap_proba_individual import shap_proba_individual, print_individual, print_reason, print_individual_waterfall
from shap_vals import shap_vals
from print_examples import print_examples
import shap

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Load the TensorBoard notebook extension
%load_ext tensorboard

pd.options.display.max_colwidth = 100


selected = stat[(stat['muncipality_code']==49)].copy()
selected_vaalidata = vaalidata[(vaalidata['Kuntanumero']==49)].copy()
post_included = post[post['muncipality_code']==49]['postcode'].copy()


selected = stat[(stat['muncipality_code']==91)].copy()
selected_vaalidata = vaalidata[(vaalidata['Kuntanumero']==91)].copy()
post_included = post[post['muncipality_code']==91]['postcode'].copy()

selected = stat[stat['area_code']=='FI1B1'].copy()
selected_vaalidata = vaalidata[vaalidata['area_code']=='FI1B1'].copy()
post_included = post[post['area_code']=='FI1B1'].copy()

selected = stat.copy()
selected_vaalidata = vaalidata.copy()
post_included = post.copy()



In [5]:
import xgboost as xgb
from sklearn.feature_selection import  mutual_info_classif


data, X, X_scale = prepare_and_scale_data(selected, pd.DataFrame(), numeric_features, categorical_features)

from create_prediction import create_prediction
values = data.sort_values(by=['Suurin_puolue numero'])['Suurin_puolue numero'].unique()
j = 0
for i in values:
    data.loc[data['Suurin_puolue numero']==i, 'Suurin_puolue numero uusi numero'] = j
    j = j + 1

data.loc[:,'Suurin_puolue numero uusi numero'] = data['Suurin_puolue numero uusi numero'].astype(int) 
target = 'Suurin_puolue numero uusi numero'

#Check
#k_selected need to have correct values, if they are too large, evrything needs to be started again
k_selected = 'all'

test_size = 0.2

#Scalers attributes
kbest_score_func = mutual_info_classif
#set values to model configuration attributes

num_class = len(data['Suurin_puolue numero uusi numero'].unique()) #how many classes we are working with

metric = 'merror'
Skfold=False
Verbose = False
testing=True
scaled = False

params = {
    # Parameters that we are going to tune.
    'objective':'multi:softmax',
    'num_class' : num_class,
    'booster' : 'gbtree',
    'eval_metric' : metric
} 

filename_model = 'xgboost_largeparty'
data, test, features_df, importance_df, model, params, y_pred, X_train, y_train, X_test, y_test = create_prediction(filename_model, path, data, data, target, kbest_score_func, metric, params, numeric_features, categorical_features, scaled=scaled, testing=testing)
data_no_use, X_scale, X = prepare_and_scale_data(data, pd.DataFrame(), numeric_features, categorical_features)


Create model
[0]	train-merror:0.18308	test-merror:0.32828
Multiple eval metrics have been passed: 'test-merror' will be used for early stopping.

Will train until test-merror hasn't improved in 50 rounds.
Stopping. Best iteration:
[27]	train-merror:0.00000	test-merror:0.25084

testing
Start with xgb.cv params: merror: 0.25925933333333334
Best xgb.cv params: max_depth 6, min_child_weight 1, merror: 0.25925933333333334
Best xgb.cv params: gamma 0.0, merror: 0.25925933333333334
Best xgb.cv params: lambda 1.0, alpha 0.0, merror: 0.25925933333333334
Best xgb.cv params: colsample_bytree 1.0, subsample 0.6, merror: 0.25883833333333334
Best xgb.cv params: eta 0.2, merror: 0.26473066666666667
Found hyperparameters with 1 rounds 
{'objective': 'multi:softmax', 'num_class': 8, 'booster': 'gbtree', 'eval_metric': 'merror', 'max_depth': 6, 'min_child_weight': 1, 'gamma': 0.0, 'lambda': 1.0, 'alpha': 0.0, 'colsample_bytree': 1.0, 'subsample': 0.6, 'eta': 0.2}

[0]	train-merror:0.22391	test-merror:0.

In [ ]:
filename_model = 'mymodel_xgboost_largeparty'
mymodel, explainer, shap_values, shap_interaction_values, shap_expected_value = shap_Xboost(filename_model, path, data, target, params, X)

data.loc[:, "Ennustettu " + target] = mymodel.predict(X)
from reset_party_number import reset_party_number
data, class_names = reset_party_number(data)

Create model


Setting feature_perturbation = "tree_path_dependent" because no background data was given.


In [ ]:
shap.summary_plot(shap_values, X,  max_display=25,class_names = class_names)

In [ ]:
pnro = '00520'
id_col = 'Postinumero'
df, X_rand, prty = shap_proba_individual(explainer, mymodel, data, target, X, id_col, pnro)


In [ ]:
print_reason(X_rand, explainer, shap_values, mymodel, 0)

In [ ]:
X_rand

In [ ]:
shap.force_plot(explainer.expected_value[7], explainer.shap_values[7,140], X_rand)

In [ ]:
shap.force_plot(shap_expected_value[1], shap_values[1][2680], X.iloc[1285], link='logit')

In [ ]:
X_rand

In [ ]:
link='logit'

In [ ]:
print_individual_waterfall(data, target, X, shap_expected_value, shap_values, id_col, pnro)

In [ ]:
party = 'KD'
idx = class_names.index(party)
shap.summary_plot(shap_values[idx], X,  max_display=15)

In [ ]:
party = 'KESK'
idx = class_names.index(party)
shap.dependence_plot("rank(0)", shap_values[idx], X)

In [ ]:
new = ['Postinumero','muncipality_name','Asukkaat yhteensä, 2018 (HE)', 'Suurin_puolue', target]
show_df, col_list = print_examples(data, X, new, target, 1, shap_values, col_num=9, n=6)
show_df

In [ ]:
shap.summary_plot(shap_interaction_values[0], X)

In [ ]:
import numpy as np
data['prop'] = np.where(data['Suurin_puolue']==data['Ennustettu Suurin_puolue nimi'],True, False)
data[(data['prop']==False) & (data['Ennustettu Suurin_puolue nimi']=='KOK')]

In [ ]:
shap.dependence_plot(
    (col_list[0], col_list[1]),
    shap_interaction_values[Party], X,
    display_features=X
)